# Documentacion de los datos

In [2]:
from DataLoader import leer_csv
from inline_sql import sql, sql_val

## eci_customer_data

In [3]:
customer_data=leer_csv("eci_customer_data")
customer_data

,client_id,phone_number,email_address,city,state,zip_code,education_level,occupation,loyalty_member,loyalty_number,loyalty_points
0,500001,NaN,n.edwards+spam@outlook.com,Spokane,WA,99201,High School,NaN,No,NaN,339
1,500002,(412) 698-7960,lisa_cook@yahoo.com,Pittsburgh,PA,15222,High School,Sales Representative,Yes,LP525082,406
2,500003,(303) 491-3774,NaN,Fort Collins,CO,80526,College Graduate,Analyst,True,950139,14
3,500004,(803) 255-0104,betty-campbell@gmail.com,Charleston,SC,29401,Some College,Supervisor,No,NaN,NaN
4,500005,(303) 596-8987,NaN,Aurora,CO,80012,Associates,NaN,Yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
801918,1521947,(216) 354-1664,demo39409@outlook.com,Akron,OH,44312,Bachelors,Analyst,True,894546,261
801919,1521948,(614) 398-0640,email38219@hotmail.com,Columbus,OH,43229,NaN,Administrative Assistant,No,NaN,NaN
801920,1521949,NaN,Helen_King@Gmail.Com,Seattle,WA,98115,Some College,Assistant,Y,LP134711,40941
801921,1521950,(918) 367-7700,email1975@hotmail.com,Norman,OK,73069,HS,Homemaker,Yes,LP892060,554


### Atributos
* client_id
* phone_number
* email_adress
* city
* state
* zip_code
* education_level
* occupation
* loyalty_member
* loyalty_number
* loyalty_points

In [5]:
customer_data.isnull().sum() 

client_id               0
phone_number        96174
email_address      144844
city                    0
state                   0
zip_code            67260
education_level    119881
occupation         144516
loyalty_member      39849
loyalty_number     361242
loyalty_points     409174
dtype: int64

Hay muchos nulls, para nuestro objetivo datos como phone_number, email_adress no nos interesan. Education level y ocupation supongo que tampoco mucho pero podrian tener algunos patrones escondidos.  Los mas preocupantes son los de loyalty, porque una de las razones del dezplazamiento de la curva de la demanda son los cambios de intereses. Trabajemos un poco más en esos datos.

In [ ]:
customer_data.drop(["phone_number","email_adress"],axis=1,inplace=True)

In [12]:
#Parecen variables categoricas pero comprobemoslo
consulta=sql^"""Select DISTINCT loyalty_member from customer_data"""
consulta #mala calidad de datos

,loyalty_member
0,Yes
1,Y
2,0
3,1
4,No
5,N
6,None
7,True
8,False


Del loyalty number solo nos interesa si lo tiene o no. Podemos codificar eso entre 1 si lo tiene y 0 sino

In [24]:
customer_data.info() #son todo columnas string

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801923 entries, 0 to 801922
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   client_id        801923 non-null  int64 
 1   phone_number     705749 non-null  object
 2   email_address    657079 non-null  object
 3   city             801923 non-null  object
 4   state            801923 non-null  object
 5   zip_code         734663 non-null  object
 6   education_level  682042 non-null  object
 7   occupation       657407 non-null  object
 8   loyalty_member   762074 non-null  object
 9   loyalty_number   440681 non-null  object
 10  loyalty_points   392749 non-null  object
dtypes: int64(1), object(10)
memory usage: 67.3+ MB


In [33]:
#customer_data["loyalty_points"] = customer_data["loyalty_points"].astype(float) #hay cosas que no son numeros.
print("ERROOOOR STR EN UNA COLUMNA DE NUMEROS")

ERROOOOR STR EN UNA COLUMNA DE NUMEROS


In [4]:
consulta=sql^"""SELECT DISTINCT education_level, COUNT(client_id) FROM customer_data GROUP BY education_level"""
consulta

,education_level,count(client_id)
0,Law Degree,3469
1,Some High School,13526
2,Bachelor's Degree,109273
3,Associate Degree,40906
4,4-year degree,13631
5,JD,3403
6,High School Diploma,27451
7,PhD,13489
8,Graduate Degree,10219
9,Some College,95355


Podriamos resumir estas 27 categorias distintas en solo 5:
* Secundario_incompleto
* Secundario_completo
* Universitario_incompleto
* Universitario_completo
* Post universitario
Las cuales podrian ser una feature ordinal.

## eci_product_groups

In [12]:
products_groups=leer_csv("eci_product_groups")
products_groups

,sku,product_name,price_group_id,price_group_name,group_type
0,BOOEDTE001,PageTurn Essential Textbooks,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
1,BOOEDTE002,Bookworm Advanced Textbooks,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
2,BOOEDTE004,PageTurn Essential Textbooks,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
3,CLOMESH002,DressRight Shirts Collection,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
4,CLOMESH005,StyleX Relaxed Shirts,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
...,...,...,...,...,...
75,SPOFIEQ005,ActiveLife Active Equipment,WELLNESS_BUNDLE_01,Wellness & Self-Care,Lifestyle
76,SPOFIEQ007,FitZone Active Equipment,WELLNESS_BUNDLE_01,Wellness & Self-Care,Lifestyle
77,SPOFISU002,AthletePro Sport Supplements,WELLNESS_BUNDLE_01,Wellness & Self-Care,Lifestyle
78,SPOFISU004,FitZone Pro Supplements,WELLNESS_BUNDLE_01,Wellness & Self-Care,Lifestyle


### Atributos
* sku
* prodouct_name
* price_group_id
* price_group_name
* group_type

## eci_product_master

In [13]:
product_master=leer_csv("eci_product_master")
product_master

,sku,product_name,category,group,subgroup,brand,base_price,initial_ticket_price,costos
0,ELECOLA001,TechPro Laptops Smart 2025,Electronics,Computing,Laptops,TechPro,284.94,355.14,225.84
1,ELECOLA002,Electra Laptops Ultra Pro,Electronics,Computing,Laptops,Electra,311.83,352.74,209.99
2,ELECOLA003,CircuitCity Laptops Air 2024,Electronics,Computing,Laptops,CircuitCity,246.96,301.35,192.61
3,ELECOLA004,TechPro Laptops Elite 2025,Electronics,Computing,Laptops,TechPro,324.96,420.49,212.23
4,ELECOLA005,CircuitCity Laptops Pro S,Electronics,Computing,Laptops,CircuitCity,339.88,386.23,231.75
...,...,...,...,...,...,...,...,...,...
856,TOYGAPU002,PlayTime Creative Puzzles,Toys,Games,Puzzles,PlayTime,33.63,39.72,19.70
857,TOYGAPU003,KidJoy Creative Puzzles,Toys,Games,Puzzles,KidJoy,34.86,41.67,16.19
858,TOYGAPU004,ImagineThat Fun Puzzles,Toys,Games,Puzzles,ImagineThat,26.70,34.60,14.10
859,TOYGAPU005,ToyWorld Fun Puzzles,Toys,Games,Puzzles,ToyWorld,31.23,38.40,14.73


### Atributos
* sku
* product_name
* category
* group
* subgroup
* brand
* base_price
* base_price
* initial_ticket_price
* costos

## eci_stores

In [14]:
stores=leer_csv("eci_stores")
stores

,store_id,brand,store_name,address1,address2,city,state,zip,opendate,closedate,store_type,region
0,S00001,PrimeGoods,PrimeGoods - Hillsboro,5735 NE Washington Pl,NaN,Hillsboro,OR,74820,2015-01-10 00:00:00,NaN,Express,West
1,S00002,MegaShop,MegaShop Indianapolis Eastside,3557 Cedar Ct,Unit 89,Indianapolis,IN,33483-1775,2012-03-23 00:00:00,NaN,Express,Midwest
2,STORE0003,MarketExpress,MarketExpress - Las Vegas,7870 W Church Ave,NaN,Las Vegas,NV,49099,2021-09-23 00:00:00,NaN,Outlet,West
3,STORE0004,MegaShop,MegaShop - Oklahoma City,3105 N Washington Pl,NaN,Oklahoma City,OK,80592,06/30/2015,NaN,Mall,Southwest
4,S00005,PrimeGoods,PrimeGoods - Charlotte,8756 SW Maple Ln,Suite 848,Charlotte,NC,32299,2019-06-10 00:00:00,NaN,Express,Southeast
...,...,...,...,...,...,...,...,...,...,...,...,...
152,S00153,MegaShop,MegaShop - Oklahoma City,3894 Park Pl,NaN,Oklahoma City,OK,86274,2020-04-01 00:00:00,2033-03-29 00:00:00,Boutique,Southwest
153,S00154,BudgetShop,BudgetShop - Newark,5992 View Rd,NaN,Newark,NJ,61756,2009-01-05 00:00:00,NaN,Outlet,Northeast
154,S00155,EssentialsPlus,EssentialsPlus - Columbia,6240 N River Ave,NaN,Columbia,SC,99211-2505,2015-05-20 00:00:00,NaN,Mall,Southeast
155,S00156,FamilyStore,FamilyStore - Naperville,4679 Market Dr,NaN,Naperville,IL,86674,2014-12-23 00:00:00,NaN,Street,Midwest


### Atributos
* stores_id
* brand
* stores_name
* adress1
* adress2
* city
* state
* zip
* opendate
* closedate
* store_type
* region

La verdad hay datos poco relevantes para nuestro objetivo:
* adress 1 y 2
* opendate
* closedate
* store_type
* store_name
* brand
* zip

## eci_stores_clusters

In [10]:
stores_clusters=leer_csv("eci_stores_clusters.csv")
stores_clusters #algo anda raro en el store id pero supongo que esta bien porque en el archivo de test tambien anda raro

,store_id,brand,store_name,cluster
0,S00001,PrimeGoods,PrimeGoods - Hillsboro,Cluster_West_Coast
1,S00002,MegaShop,MegaShop Indianapolis Eastside,Midwest_Cluster
2,STORE0003,MarketExpress,MarketExpress - Las Vegas,Mountain_Cluster
3,STORE0004,MegaShop,MegaShop - Oklahoma City,Texas_Cluster
4,S00005,PrimeGoods,PrimeGoods - Charlotte,NC_Cluster
...,...,...,...,...
152,S00153,MegaShop,MegaShop - Oklahoma City,Texas_Cluster
153,S00154,BudgetShop,BudgetShop - Newark,NJ_Cluster
154,S00155,EssentialsPlus,EssentialsPlus - Columbia,SC_Cluster
155,S00156,FamilyStore,FamilyStore - Naperville,Midwest_Cluster


### Atributos
* store_id
* brand
* store_name
* cluster

Hay atributos poco relevantes como:
* brand
* store_name
  
Podriamos hacer 2 enfoques trabajar con la informacion obtenida a partir de los clusters brindados o trabajando store por store

## eci_transactions

In [16]:
transactions=leer_csv("eci_transactions")
transactions

,transaction_id,date,store_id,sku,quantity,price,total_sales,subgroup,store_subgroup_date_id
0,17185,2021-01-01,S00148,CLOMEJA001,1.0,52.80,52.80,Jackets,S00148_Jackets_2021-01-01
1,4108,2021-01-01,S00035,CLOCHIN014,1.0,55.25,55.25,Infants,S00035_Infants_2021-01-01
2,2400,2021-01-01,S00020,BEAHASH005,1.0,40.19,40.19,Shampoo,S00020_Shampoo_2021-01-01
3,7978,2021-01-01,S00068,TOYGABO004,1.0,35.41,35.41,Board Games,S00068_Board Games_2021-01-01
4,6946,2021-01-01,S00059,BEAHASH006,1.0,38.28,38.28,Shampoo,S00059_Shampoo_2021-01-01
...,...,...,...,...,...,...,...,...,...
19004754,19001120,2023-12-31,S00122,GROSNCH011,1.0,10.12,10.12,Chips,S00122_Chips_2023-12-31
19004755,19000886,2023-12-31,S00120,GROSNNU008,1.0,11.40,11.40,Nuts,S00120_Nuts_2023-12-31
19004756,18989298,2023-12-31,STORE0004,CLOMEJA005,1.0,56.86,56.86,Jackets,STORE0004_Jackets_2023-12-31
19004757,18992918,2023-12-31,S00039,CLOCHBO011,1.0,57.24,57.24,Boys,S00039_Boys_2023-12-31


### Atributos
* trasanction_id
* date
* store_id
* sku
* quantity
* price
* total_sales
* subgroup
* store_subgroup_date_id